# Project: Kickstarter Project

## Exploring Kickstarter Data and Identifying Key Factors for Project Success

In [2]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from patsy import dmatrices
import statsmodels.api as sm;
from datetime import *
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

In [3]:
# read in csv
df_og = pd.read_csv('ks-projects-201801.csv')

## Introduction:

## Data Assessment:

In [121]:
# visual assessment
df_og.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


* Does not look like we'll be needing the ID column for analysis
* As usd conversion is already done for us, we won't be needing currency either (we still have country recorded)
* Extract live_time from deadline - launched
* Remove goal as well since we have usd_goal_real
* Discrepancy between usd pledged and usd_pledged_real (Row #2) - We will take usd_pledged_real (converted by fixer.io api) over KS conversions due to errors

In [122]:
df_og.category.value_counts()

Product Design       22314
Documentary          16139
Music                15727
Tabletop Games       14180
Shorts               12357
Video Games          11830
Food                 11493
Film & Video         10108
Fiction               9169
Fashion               8554
Nonfiction            8318
Art                   8253
Apparel               7166
Theater               7057
Technology            6930
Rock                  6758
Children's Books      6756
Apps                  6345
Publishing            6018
Webseries             5762
Photography           5752
Indie Rock            5657
Narrative Film        5188
Web                   5153
Comics                4996
Crafts                4664
Country & Folk        4451
Design                4199
Hip-Hop               3912
Hardware              3663
                     ...  
Couture                275
Blues                  268
Animals                255
Fabrication Tools      250
Makerspaces            238
Printing               238
M

In [123]:
df_og.main_category.value_counts()

Film & Video    63585
Music           51918
Publishing      39874
Games           35231
Technology      32569
Design          30070
Art             28153
Food            24602
Fashion         22816
Theater         10913
Comics          10819
Photography     10779
Crafts           8809
Journalism       4755
Dance            3768
Name: main_category, dtype: int64

* Naming convention for main cateogy and category is confusing. Rename category as sub_category

In [124]:
df_og.state.value_counts()

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64

In [125]:
df_og.currency.value_counts()

USD    295365
GBP     34132
EUR     17405
CAD     14962
AUD      7950
SEK      1788
MXN      1752
NZD      1475
DKK      1129
CHF       768
NOK       722
HKD       618
SGD       555
JPY        40
Name: currency, dtype: int64

In [126]:
df_og.country.value_counts()

US      292627
GB       33672
CA       14756
AU        7839
DE        4171
N,0"      3797
FR        2939
IT        2878
NL        2868
ES        2276
SE        1757
MX        1752
NZ        1447
DK        1113
IE         811
CH         761
NO         708
HK         618
BE         617
AT         597
SG         555
LU          62
JP          40
Name: country, dtype: int64

In [127]:
string = 'N,0"'
df_no = df_og.query('country == @string')

In [128]:
df_no.tail()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
378233,997971307,"EQUUS International Film Festival®, all-equine...",Film & Video,Film & Video,USD,2015-02-03,7500.0,2014-12-05 04:19:14,10.0,undefined,0,"N,0""",NaN,10.00,7500.00
378303,998319149,Emily 2050 - Short Film,Film & Video,Film & Video,CAD,2014-05-23,3000.0,2014-04-08 00:30:09,3102.0,undefined,0,"N,0""",NaN,2845.61,2752.04
378434,9988744,Matthew Stephens Music,Music,Music,USD,2016-02-05,5000.0,2016-01-06 21:59:23,235.0,undefined,0,"N,0""",NaN,235.00,5000.00
378585,999610349,Lady Vendredi: Afrofuturist concept 12 inch EP,Music,Music,GBP,2015-10-19,2000.0,2015-09-21 22:33:18,2125.0,undefined,0,"N,0""",NaN,3273.36,3080.81
378588,999621278,Jeune auteur a besoin de vous!,Publishing,Publishing,EUR,2015-11-20,1600.0,2015-10-21 21:28:13,0.0,undefined,0,"N,0""",NaN,0.00,1710.08


* Fix Typo in Country: N,0"
  * Upon taking a closer look at rows with country listed as "N,0"" and each of the projects on kickstarter.com, there was no single country that all of these projects belonged to. I assume that it is being recorded as NA. Not all hope is lost though, I found that the currency is the perfect indicator as to which country the project belongs to. Therefore, we will need to keep the currency column to fill in N,0"s and drop it later on

In [129]:
df_og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
ID                  378661 non-null int64
name                378657 non-null object
category            378661 non-null object
main_category       378661 non-null object
currency            378661 non-null object
deadline            378661 non-null object
goal                378661 non-null float64
launched            378661 non-null object
pledged             378661 non-null float64
state               378661 non-null object
backers             378661 non-null int64
country             378661 non-null object
usd pledged         374864 non-null float64
usd_pledged_real    378661 non-null float64
usd_goal_real       378661 non-null float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


In [130]:
type(df_og.deadline[0])

str

In [131]:
type(df_og.launched[0])

str

* change both deadline and launch date to datetime object

## Wrangling

In [260]:
df = df_og.copy()

In [261]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [262]:
# selecting columns I want to use for analysis
df = df[['name', 'main_category', 'category', 'currency', 'country', 'launched', 'deadline', 'state', 'backers', 'usd_pledged_real', 'usd_goal_real']]

In [263]:
df.head()

,name,main_category,category,currency,country,launched,deadline,state,backers,usd_pledged_real,usd_goal_real
0,The Songs of Adelaide & Abullah,Publishing,Poetry,GBP,GB,2015-08-11 12:12:28,2015-10-09,failed,0,0.0,1533.95
1,Greeting From Earth: ZGAC Arts Capsule For ET,Film & Video,Narrative Film,USD,US,2017-09-02 04:43:57,2017-11-01,failed,15,2421.0,30000.00
2,Where is Hank?,Film & Video,Narrative Film,USD,US,2013-01-12 00:20:50,2013-02-26,failed,3,220.0,45000.00
3,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,US,2012-03-17 03:24:11,2012-04-16,failed,1,1.0,5000.00
4,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,US,2015-07-04 08:35:03,2015-08-29,canceled,14,1283.0,19500.00


In [264]:
# renaming columns
df.columns = ['name', 'main_cat', 'sub_cat', 'currency', 'country', 'launch_date', 'deadline', 'state', 'backers', 'usd_pledged', 'usd_goal']

In [265]:
df.head()

,name,main_cat,sub_cat,currency,country,launch_date,deadline,state,backers,usd_pledged,usd_goal
0,The Songs of Adelaide & Abullah,Publishing,Poetry,GBP,GB,2015-08-11 12:12:28,2015-10-09,failed,0,0.0,1533.95
1,Greeting From Earth: ZGAC Arts Capsule For ET,Film & Video,Narrative Film,USD,US,2017-09-02 04:43:57,2017-11-01,failed,15,2421.0,30000.00
2,Where is Hank?,Film & Video,Narrative Film,USD,US,2013-01-12 00:20:50,2013-02-26,failed,3,220.0,45000.00
3,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,US,2012-03-17 03:24:11,2012-04-16,failed,1,1.0,5000.00
4,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,US,2015-07-04 08:35:03,2015-08-29,canceled,14,1283.0,19500.00


In [266]:
# Converting launch date and deadline to datetime object
df.launch_date = pd.to_datetime(df.launch_date)
df.deadline = pd.to_datetime(df.deadline)

# Creating project live time column by calculating difference between launch and deadline
df['live_time'] = df.deadline - df.launch_date

In [267]:
df.head()

,name,main_cat,sub_cat,currency,country,launch_date,deadline,state,backers,usd_pledged,usd_goal,live_time
0,The Songs of Adelaide & Abullah,Publishing,Poetry,GBP,GB,2015-08-11 12:12:28,2015-10-09,failed,0,0.0,1533.95,58 days 11:47:32
1,Greeting From Earth: ZGAC Arts Capsule For ET,Film & Video,Narrative Film,USD,US,2017-09-02 04:43:57,2017-11-01,failed,15,2421.0,30000.00,59 days 19:16:03
2,Where is Hank?,Film & Video,Narrative Film,USD,US,2013-01-12 00:20:50,2013-02-26,failed,3,220.0,45000.00,44 days 23:39:10
3,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,US,2012-03-17 03:24:11,2012-04-16,failed,1,1.0,5000.00,29 days 20:35:49
4,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,US,2015-07-04 08:35:03,2015-08-29,canceled,14,1283.0,19500.00,55 days 15:24:57


In [268]:
# separate data into 2. One with countries listed as N,0" and the rest so we can fix and concatenate afterwards
df_no = df.query('country == @string')
df = df.query('country != @string')

In [269]:
# taking a look at currencies present 
df_no.currency.value_counts()

USD    2738
GBP     460
CAD     206
EUR     186
AUD     111
SEK      31
NZD      28
DKK      16
NOK      14
CHF       7
Name: currency, dtype: int64

In [270]:
# calculating proportion of countries with EURO as their main currency in df (not df_no)
n_eur = df.query('currency == "EUR"').country.count()
df.query('currency == "EUR"').country.value_counts() / n_eur

DE    0.242232
FR    0.170684
IT    0.167141
NL    0.166560
ES    0.132180
IE    0.047099
BE    0.035833
AT    0.034671
LU    0.003601
Name: country, dtype: float64

In [271]:
print(n_eur)

17219


n = 17,219

Approximate % of countries in each projects using EUR as main currency:

* DE - 24
* FR - 17
* IT - 17
* NL - 17
* ES - 13
* IE - 5
* BE - 3
* AT - 3
* LU - 1

In [272]:
for i in df_no.index:
    # grab currency data
    ccy = df_no.get_value(i, 'currency')
    
    # USA
    if ccy == 'USD':
        df_no.set_value(i, 'country', 'US')
    
    # UK
    elif ccy == 'GBP':
        df_no.set_value(i, 'country', 'UK')
    
    # Canada    
    elif ccy == 'CAD':
        df_no.set_value(i, 'country', 'CA')
    
    # Euros - Since more than 1 country uses EUR, I will distribute each project into a random country following the proportion above
    elif ccy == 'EUR':
        x = np.random.randint(1, 100)
        
        if (x <= 24):
            df_no.set_value(i, 'country', 'DE')
        
        elif (x > 24) & (x <= 41):
            df_no.set_value(i, 'country', 'FR')
        
        elif (x > 41) & (x <= 58):
            df_no.set_value(i, 'country', 'IT')
            
        elif (x > 58) & (x <= 75):
            df_no.set_value(i, 'country', 'NL')
        
        elif (x > 75) & (x <= 88):
            df_no.set_value(i, 'country', 'ES')
        
        elif (x > 88) & (x <= 93):
            df_no.set_value(i, 'country', 'IE')
        
        elif (x > 93) & (x <= 96):
            df_no.set_value(i, 'country', 'BE')
            
        elif (x > 96) & (x <= 99):
            df_no.set_value(i, 'country', 'AT')
            
        elif (x == 100):
            df_no.set_value(i, 'country', 'LU')      
    
    # Australia
    elif ccy == 'AUD':
        df_no.set_value(i, 'country', 'AU')
    
    # Sweden
    elif ccy == 'SEK':
        df_no.set_value(i, 'country', 'SE')
       
    # New Zealand
    elif ccy == 'NZD':
        df_no.set_value(i, 'country', 'NZ')
    
    # Denmark
    elif ccy == 'DKK':
        df_no.set_value(i, 'country', 'DK')
     
    # Norway
    elif ccy == 'NOK':
        df_no.set_value(i, 'country', 'NO')
    
    # Switzerland
    elif ccy == 'CHF':
        df_no.set_value(i, 'country', 'CH')

C:\Users\paul.seo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\paul.seo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
C:\Users\paul.seo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\paul.seo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import

In [273]:
df_no.query('currency == "EUR"').country.value_counts()

DE    40
FR    39
NL    35
IT    30
ES    24
IE    12
BE     3
AT     3
Name: country, dtype: int64

Distribution here resembles the bigger sample from df (n = 17,219), which is a great sign!

In [274]:
# concatenate df_no back into df
df = pd.concat([df, df_no], ignore_index = True)

In [275]:
df.tail()

,name,main_cat,sub_cat,currency,country,launch_date,deadline,state,backers,usd_pledged,usd_goal,live_time
378656,"EQUUS International Film Festival®, all-equine...",Film & Video,Film & Video,USD,US,2014-12-05 04:19:14,2015-02-03,undefined,0,10.00,7500.00,59 days 19:40:46
378657,Emily 2050 - Short Film,Film & Video,Film & Video,CAD,CA,2014-04-08 00:30:09,2014-05-23,undefined,0,2845.61,2752.04,44 days 23:29:51
378658,Matthew Stephens Music,Music,Music,USD,US,2016-01-06 21:59:23,2016-02-05,undefined,0,235.00,5000.00,29 days 02:00:37
378659,Lady Vendredi: Afrofuturist concept 12 inch EP,Music,Music,GBP,UK,2015-09-21 22:33:18,2015-10-19,undefined,0,3273.36,3080.81,27 days 01:26:42
378660,Jeune auteur a besoin de vous!,Publishing,Publishing,EUR,FR,2015-10-21 21:28:13,2015-11-20,undefined,0,0.00,1710.08,29 days 02:31:47


I'm noticing some of the kickstarter projects that had backers (usd_pledged > 0) are showing up with 0 backers. I need to investigate this further:

In [276]:
df.query('backers == 0').usd_pledged.count()

55609

In [277]:
df.query('backers == 0').usd_pledged.value_counts()

0.00       52527
1.00         123
25.00         32
10.00         28
2.00          24
100.00        24
50.00         22
5.00          20
20.00         14
30.00         13
6.00          11
200.00        10
2000.00       10
51.00          9
3000.00        8
21.00          8
101.00         8
1500.00        8
120.00         7
5000.00        7
230.00         7
110.00         7
150.00         6
75.00          6
26.00          6
11.00          6
61.00          6
3.00           6
15.00          6
31.00          5
           ...  
4497.90        1
2975.00        1
423.00         1
2004.27        1
1.60           1
8830.00        1
4650.00        1
35.60          1
9855.00        1
218.00         1
440.00         1
704.00         1
69.86          1
265.66         1
1515.00        1
409.00         1
281.76         1
47.80          1
811.00         1
1774.43        1
537.42         1
2486.23        1
32.58          1
10.23          1
39.26          1
829.05         1
6376.00        1
32.06         

Out of 55,609 rows with 0 backers, 52,527 is correctly recorded as having received $0 in pledges. Since we cannot extract backer numbers from existing data, the only solution is manually fixing 3,000 projects, which is not practical at this point with our data already being a good size (~380k). I will remove these rows before moving on further:

In [278]:
# Drop rows specified above, and reset index
df.drop(df[(df.backers == 0) & (df.usd_pledged != 0)].index, inplace = True)
df.reset_index(drop = True, inplace = True)

In [279]:
df.tail()

,name,main_cat,sub_cat,currency,country,launch_date,deadline,state,backers,usd_pledged,usd_goal,live_time
375574,Bydlo - A Novel,Publishing,Publishing,GBP,UK,2015-04-02 12:51:16,2015-06-01,undefined,0,0.0,4599.39,59 days 11:08:44
375575,"DEVELOP MAGAZINE FOR THE ARTS, MUSIC, AND DESI...",Publishing,Publishing,USD,US,2015-01-29 17:43:13,2015-02-28,undefined,0,0.0,5000.00,29 days 06:16:47
375576,"""Songwriter assistance"" service",Music,Music,CAD,CA,2014-11-22 00:56:04,2014-12-22,undefined,0,0.0,4313.32,29 days 23:03:56
375577,The World Needs More Big Thinkers,Publishing,Publishing,USD,US,2015-07-30 03:01:31,2015-09-13,undefined,0,0.0,8000.00,44 days 20:58:29
375578,Jeune auteur a besoin de vous!,Publishing,Publishing,EUR,FR,2015-10-21 21:28:13,2015-11-20,undefined,0,0.0,1710.08,29 days 02:31:47


In [280]:
df.query('backers == 0').usd_pledged.value_counts()

0.0    52527
Name: usd_pledged, dtype: int64

All rows with 0 backers now have $0 in pledged amount.

In [281]:
len(df)

375579

In [282]:
list(df.columns.values)

['name',
 'main_cat',
 'sub_cat',
 'currency',
 'country',
 'launch_date',
 'deadline',
 'state',
 'backers',
 'usd_pledged',
 'usd_goal',
 'live_time']

In [283]:
# re-ordering columns and now dropping currency
df = df[['name',
 'main_cat',
 'sub_cat',
 'country',
 'launch_date',
 'deadline',
 'live_time',
 'state',
 'backers',
 'usd_pledged',
 'usd_goal']]

In [284]:
# dropping NAs and duplicates
df.drop_duplicates(inplace = True)
df.dropna(inplace = True)

In [285]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375575 entries, 0 to 375578
Data columns (total 11 columns):
name           375575 non-null object
main_cat       375575 non-null object
sub_cat        375575 non-null object
country        375575 non-null object
launch_date    375575 non-null datetime64[ns]
deadline       375575 non-null datetime64[ns]
live_time      375575 non-null timedelta64[ns]
state          375575 non-null object
backers        375575 non-null int64
usd_pledged    375575 non-null float64
usd_goal       375575 non-null float64
dtypes: datetime64[ns](2), float64(2), int64(1), object(5), timedelta64[ns](1)
memory usage: 34.4+ MB


In [286]:
df.head(10)

,name,main_cat,sub_cat,country,launch_date,deadline,live_time,state,backers,usd_pledged,usd_goal
0,The Songs of Adelaide & Abullah,Publishing,Poetry,GB,2015-08-11 12:12:28,2015-10-09,58 days 11:47:32,failed,0,0.00,1533.95
1,Greeting From Earth: ZGAC Arts Capsule For ET,Film & Video,Narrative Film,US,2017-09-02 04:43:57,2017-11-01,59 days 19:16:03,failed,15,2421.00,30000.00
2,Where is Hank?,Film & Video,Narrative Film,US,2013-01-12 00:20:50,2013-02-26,44 days 23:39:10,failed,3,220.00,45000.00
3,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,US,2012-03-17 03:24:11,2012-04-16,29 days 20:35:49,failed,1,1.00,5000.00
4,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,US,2015-07-04 08:35:03,2015-08-29,55 days 15:24:57,canceled,14,1283.00,19500.00
5,Monarch Espresso Bar,Food,Restaurants,US,2016-02-26 13:38:27,2016-04-01,34 days 10:21:33,successful,224,52375.00,50000.00
6,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,US,2014-12-01 18:30:44,2014-12-21,19 days 05:29:16,successful,16,1205.00,1000.00
7,Chaser Strips. Our Strips make Shots their B*tch!,Food,Drinks,US,2016-02-01 20:05:12,2016-03-17,44 days 03:54:48,failed,40,453.00,25000.00
8,SPIN - Premium Retractable In-Ear Headphones w...,Design,Product Design,US,2014-04-24 18:14:43,2014-05-29,34 days 05:45:17,canceled,58,8233.00,125000.00
9,STUDIO IN THE SKY - A Documentary Feature Film...,Film & Video,Documentary,US,2014-07-11 21:55:48,2014-08-10,29 days 02:04:12,canceled,43,6240.57,65000.00


In [287]:
# saving cleaned data to resume later
df.to_csv('temp.csv', index = False)